In [ ]:
# SONG RECOMMENDER

In [99]:
%conda install streamlit

Retrieving notices: ...working... done
done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



## Package Plan ##

  environment location: /Users/minhnguyen/anaconda3

  added / updated specs:
    - streamlit


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-5.2.0               |     pyhd8ed1ab_0         447 KB  conda-forge
    blinker-1.7.0              |     pyhd8ed1ab_0          17 KB  conda-forge
    cachetools-5.3.2           |     pyhd8ed1ab_0          14 KB  conda-forge
    gitdb-4.0.11               |     pyhd8ed1ab_0          52 KB  conda-forge
    gitpython-3.1.41           |     pyhd8ed1ab_0         147 KB  co

In [198]:
%%writefile song_recommender.py
#import libraries
import requests
import pandas as pd
import numpy as np 
import pickle

import sys
sys.path.append('/Users/minhnguyen/IronHack2023-2024/Bootcamp/')
from config_2 import *

import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
from time import sleep
import streamlit as st
#Initialize SpotiPy with user credentias #
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=Client_ID, client_secret=Client_Secret))
# import libraries
#from recommend_function_2 import *

# load song_db
song_db = pd.read_csv('song_db_cluster.csv')

# split list of song ids:
def chunks (song_ids, n:int =50)-> list:
    """
    Divides a sequence of song IDs into chunks of a specified size.

    Parameters:
    - song_ids (list or pandas.DataFrame): The sequence of song IDs to be divided into chunks.
      It can be either a list or a pandas DataFrame.
    - n (int, optional): The desired size of each chunk. Default is 50.

    Returns:
    - list: A list containing chunks of song IDs, where each chunk has a maximum size of 'n'.

    Note:
    - If 'song_ids' is a list, the chunks are created using list slicing.
    - If 'song_ids' is a pandas DataFrame, the chunks are created using DataFrame row slicing.
    - If 'song_ids' is smaller than 'n', a single chunk containing all elements is returned.
    """
    if len(song_ids) > n:
        if type(song_ids) == list:
            chunks = [song_ids[x:x+n] for x in range(0, len(song_ids), n)]
            return chunks
        elif type(song_ids) == pd.DataFrame:
            chunks = [song_ids.iloc[x:x+n,] for x in range(0, len(song_ids), n)]
            return chunks
        else:
            pass
        
    else:
        chunks = [song_ids]
        return chunks


def song_info_spotify(title:str, artist:str ='', limit:int = 1):

    track_id_list = []
    track_name_list = []
    artist_name_list = []
    href_list = []
    popularity_list = []

    if artist == '':
        try:
            result = sp.search(q=f"track:{title}", limit=limit)
            for i in range(0,limit):
                track_id = result['tracks']['items'][i]['id']
                track_id_list.append(track_id)
                track_name = result['tracks']['items'][i]['name']
                track_name_list.append(track_name)
                href = result['tracks']['items'][i]['href']
                href_list.append(href)
                artist_name = result['tracks']['items'][i]['artists'][0]['name']
                artist_name_list.append(artist_name)
                popularity = result['tracks']['items'][i]['popularity']
                popularity_list.append(popularity)

        except:
            print('Song not found')
            track_id_list.append('None')
            track_name_list.append('None')
            href_list.append('None')
            artist_name_list.append("None")
            popularity_list.append('None')

    else:
        try:
            result = sp.search(q=f"track:{title} artist:{artist}", limit=limit)
            for i in range(0,limit):
                track_id = result['tracks']['items'][i]['id']
                track_id_list.append(track_id)
                track_name = result['tracks']['items'][i]['name']
                track_name_list.append(track_name)
                href = result['tracks']['items'][i]['href']
                href_list.append(href)
                artist_name = result['tracks']['items'][i]['artists'][0]['name']
                artist_name_list.append(artist_name)
                popularity = result['tracks']['items'][i]['popularity']
                popularity_list.append(popularity)

        except:
            print('Song not found')
            track_id_list.append('None')
            track_name_list.append('None')
            href_list.append('None')
            artist_name_list.append("None")
            popularity_list.append('None')

    track_info = pd.DataFrame({'song_id':track_id_list, 'track_name': track_name_list, 'artist_name': artist_name_list, 'track_href': href_list, 'popularity': popularity_list})

    return track_info

# function for getting audio features
def get_audio_features (list:list):
    
    sublists = chunks(list,100)
    audio_features_dict ={'danceability':[], 'energy':[], 'key':[], 'loudness':[], 'mode':[], 'speechiness':[], 'acousticness':[],'instrumentalness':[], 'liveness':[], 'valence':[], 'tempo':[], 'type':[], 'id':[], 'uri':[], 'track_href':[], 'analysis_url':[], 'duration_ms':[], 'time_signature':[]}
    for index,list in enumerate(sublists):
        #print(f"Retrieving audio_features from chunk {index}")
        # get audio_features
        try:
            audio_features = sp.audio_features(list)
            for feature in audio_features:
                for key in audio_features_dict:
                    audio_features_dict[key].append(feature[key])
            #audio_features['song_id'] = song_id # add dict item with key 'song_id' and value song_id            
            #audio_features_list.append(audio_features)
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:
                retry_after = int(e.response.headers.get('Retry-After', 1))
                print(f"Rate limited. Retrying after {retry_after} seconds.")
                sleep(retry_after + 1)
                continue
            else:
                raise
        except Exception as e:
            print(f"Failed to get audio features for some track IDs: {e}")

        #print("sleep a bit before getting the next chunk")
        print('Processing...')  
        sleep(10)

    audio_features_df = pd.DataFrame(audio_features_dict)
    return audio_features_df

# function to add audio features to the track info dataframe
def add_audio_features (df1, df2, left_col, right_col, how = 'inner' ):
   
    extended_df = pd.merge(df1, df2, left_on=left_col, right_on=right_col, how = how)
    return extended_df

# function to get recommendation from the song_db
def recommendation(song_db, user_song_cluster, hot_value:str = None, num_recommendations=5):
    if hot_value != None:
        recommendation = song_db[(song_db['cluster'] == int(user_song_cluster)) & (song_db['hotness'] == hot_value)].sample(num_recommendations)
    else: 
        recommendation = song_db[(song_db['cluster'] == int(user_song_cluster))].sample(num_recommendations)

    recommendation = recommendation[['track_name', 'artist_name', 'track_href']]
    sleep(20)
    print("\n Here are recommended songs and the link to spotify for you to try out: \n")
    print()
    print(recommendation)
    
# get Scaler and model
with open( "song_cluster_scaled.pkl","rb") as file:
    scaler= pickle.load(file)
    
with open( "KMeans_cluster.pkl","rb") as file:
    model= pickle.load(file)

choice = 'yes'
while choice == 'yes':

# ask for user inputs:
    track_name = str(input("Please enter your favourite's song: "))
    artist_name = str(input("Please enter the artist name (press enter to skip): "))

#from get_feature_update import *
    song_info = song_info_spotify(track_name, artist_name, 5)
# output the spotify search result of 5 songs with track_name with or without an artist_name
    print('Processing ... \n')
    print()
    print('we retrieved the below songs from Spotify: \n')
    print()
    print(song_info[['track_name', 'artist_name', 'popularity']])
    sleep(5)
    print()
    print('\n Please choose which song you would like to search: ')

# get choice from user
    choice = int(input("Please enter the number of the song you would like to search:"))
    print()
    print('\n Processing ... \n')
# set selected track from user input
    selected_track_id = [song_info.iloc[choice, 0]]

# get audio features for the selected track
    song_audio = get_audio_features(selected_track_id)

# calling function to add audio features to the selected track info dataframe
    user_song_final = add_audio_features(song_info, song_audio, 'song_id', 'id')
# get the df with selected audio features for cluster prediction
    user_song_cluster_df = user_song_final[['danceability', 'energy', 'acousticness', 'key', 'valence']]


# perform scale data and cluster prediction for selected track
    user_song_cluster_scaled = scaler.transform(user_song_cluster_df)
    user_song_cluster_scaled_df = pd.DataFrame(user_song_cluster_scaled, columns = user_song_cluster_df.columns)
    user_song_cluster = model.predict(user_song_cluster_scaled_df)

# check in which group of 'hotness' the selected track is, and get recommendation from song_db with the same cluster and the same hotness
# output recommendation to user, and promtp for new input
    if song_info.iloc[choice, 0] in song_db[song_db['hotness'] == 'yes']['song_id'].tolist():
        recommendation = recommendation(song_db, user_song_cluster, 'yes')
        print('\n What do you think?')
        print()
        sleep(10)
        print('\n You enjoyed and would like to have another recommendation from us?\n ')
        sleep(10)
        print()
        choice = str(input("If yes, please enter 'yes' to provide a song name! \n If not, please enter 'no' to escape"))
        if choice == 'yes':
            continue
        else:
            break
    elif song_info.iloc[choice, 0] in song_db[song_db['hotness'] == 'no']['song_id'].tolist():
        recommendation = recommendation(song_db, user_song_cluster, 'no')
        print('\n What do you think?')
        print()
        sleep(10)
        print('\n You enjoyed and would like to have another recommendation from us?\n ')
        sleep(10)
        print()
        choice = str(input("If yes, please enter 'yes' to provide a song name! \n If not, please enter 'no' to escape"))
        if choice == 'yes':
            continue
        else:
            break
    else:
        print('\n We could not process the "hotness" of this song at this moment yet.\n' )
        print('\n But we think you might enjoy these songs still! \n' )
        recommendation = recommendation(song_db, user_song_cluster)
        print('\n What do you think?')
        print()
        sleep(10)
        print('\n You enjoyed and would like to have another recommendation from us?\n ')
        sleep(10)
        print()
        choice = str(input("If yes, please enter 'yes' to provide a song name! \n If not, please enter 'no' to escape"))
        if choice == 'yes':
            continue
        else:
            break


        
    

Overwriting song_recommender.py


In [116]:
#selected_track_id

In [119]:
#song_audio

In [122]:
#user_song_final

In [136]:
#user_song_cluster_df

,danceability,energy,acousticness,key,valence
0,0.735,0.478,0.189,2,0.947


In [126]:
#song_db[['track_name', 'artist_name']].sample(5)

In [162]:
%run song_recommender.py

we retrieved the below songs from Spotify: " please choose which song you would like to search: 
                        track_name artist_name  popularity
0                   Last Christmas       Wham!          93
1  Last Christmas - Single Version       Wham!          88
2      Last Christmas - Remastered       Wham!          85
3                   Last Christmas       Wham!          70
4                   Last Christmas       Wham!          71
Please choose which song you would like to search: 
Retrieving audio_features from chunk 0
sleep a bit before getting the next chunk
Here are recommended songs and the link to spotify for you to try out:


,track_name,artist_name,track_href
1882,not fade away,mick fleetwood,https://api.spotify.com/v1/tracks/1G1OgcGURIWs...
2049,cucurucucú paloma,willie colón,https://api.spotify.com/v1/tracks/1BUHjtsfyGTX...
805,it's the same old song,four tops,https://api.spotify.com/v1/tracks/7d1ustAIlXG7...
2396,it feels alright,black lips,https://api.spotify.com/v1/tracks/31hueWsGBgEE...
1102,this is how we do it,montell jordan,https://api.spotify.com/v1/tracks/6uQKuonTU8VK...


None

 What do you think?

 If you enjoy and would like to have another recommendation from us?


In [178]:
song_db

,track_name,artist_name,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,hotness,cluster
0,Rockin' Around The Christmas Tree,Brenda Lee,2EjXfH91m7f8HiJN1yQg97,0.589,0.472,8,-8.749,1,0.0502,0.614000,...,67.196,audio_features,2EjXfH91m7f8HiJN1yQg97,spotify:track:2EjXfH91m7f8HiJN1yQg97,https://api.spotify.com/v1/tracks/2EjXfH91m7f8...,https://api.spotify.com/v1/audio-analysis/2EjX...,126267,4,yes,3
1,All I Want For Christmas Is You,Mariah Carey,0bYg9bo50gSsH3LtXe2SQn,0.336,0.627,7,-7.463,1,0.0384,0.164000,...,150.273,audio_features,0bYg9bo50gSsH3LtXe2SQn,spotify:track:0bYg9bo50gSsH3LtXe2SQn,https://api.spotify.com/v1/tracks/0bYg9bo50gSs...,https://api.spotify.com/v1/audio-analysis/0bYg...,241107,4,yes,5
2,Jingle Bell Rock,Bobby Helms,7vQbuQcyTflfCIOu3Uzzya,0.754,0.424,2,-8.463,1,0.0363,0.643000,...,119.705,audio_features,7vQbuQcyTflfCIOu3Uzzya,spotify:track:7vQbuQcyTflfCIOu3Uzzya,https://api.spotify.com/v1/tracks/7vQbuQcyTflf...,https://api.spotify.com/v1/audio-analysis/7vQb...,130973,4,yes,3
3,Last Christmas,Wham!,2FRnf9qhLbvw8fu4IBXx78,0.735,0.478,2,-12.472,1,0.0293,0.189000,...,107.682,audio_features,2FRnf9qhLbvw8fu4IBXx78,spotify:track:2FRnf9qhLbvw8fu4IBXx78,https://api.spotify.com/v1/tracks/2FRnf9qhLbvw...,https://api.spotify.com/v1/audio-analysis/2FRn...,262960,4,yes,0
4,A Holly Jolly Christmas,Burl Ives,77khP2fIVhSW23NwxrRluh,0.683,0.375,0,-13.056,1,0.0303,0.579000,...,140.467,audio_features,77khP2fIVhSW23NwxrRluh,spotify:track:77khP2fIVhSW23NwxrRluh,https://api.spotify.com/v1/tracks/77khP2fIVhSW...,https://api.spotify.com/v1/audio-analysis/77kh...,135533,4,yes,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2802,reckless life,guns n' roses,7I2Q7AQqVInZYwOoDHZGOf,0.442,0.888,10,-8.820,0,0.0663,0.000054,...,108.024,audio_features,7I2Q7AQqVInZYwOoDHZGOf,spotify:track:7I2Q7AQqVInZYwOoDHZGOf,https://api.spotify.com/v1/tracks/7I2Q7AQqVInZ...,https://api.spotify.com/v1/audio-analysis/7I2Q...,200600,4,no,5
2803,pain and love,t. rex,5Qe7x9x8mfl2YcbYTP5yRM,0.588,0.629,0,-10.226,0,0.0318,0.055500,...,136.091,audio_features,5Qe7x9x8mfl2YcbYTP5yRM,spotify:track:5Qe7x9x8mfl2YcbYTP5yRM,https://api.spotify.com/v1/tracks/5Qe7x9x8mfl2...,https://api.spotify.com/v1/audio-analysis/5Qe7...,221440,4,no,0
2804,brave new world,greta van fleet,0HwdiT4MVRzY0OALcjq0Oh,0.341,0.848,7,-4.475,1,0.0371,0.058300,...,136.568,audio_features,0HwdiT4MVRzY0OALcjq0Oh,spotify:track:0HwdiT4MVRzY0OALcjq0Oh,https://api.spotify.com/v1/tracks/0HwdiT4MVRzY...,https://api.spotify.com/v1/audio-analysis/0Hwd...,300880,4,no,5
2805,i-76,g. love & special sauce,40c4Zo0r2HH9aKDgEsbsVA,0.643,0.713,11,-6.716,0,0.2150,0.138000,...,103.942,audio_features,40c4Zo0r2HH9aKDgEsbsVA,spotify:track:40c4Zo0r2HH9aKDgEsbsVA,https://api.spotify.com/v1/tracks/40c4Zo0r2HH9...,https://api.spotify.com/v1/audio-analysis/40c4...,225640,4,no,1
